In [1]:
import tokenizers
from tokenizers import CharBPETokenizer
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer
from pathlib import Path
import os
# the folder 'text' contains all the files
paths = [str(x) for x in Path("./data/").glob("**/*.txt")]
tokenizer = CharBPETokenizer(bert_normalizer=False)
# train the tokenizer model
tokenizer.train(paths)
# saving the tokenized data in our specified folder 
save_path = 'tokenized_data'
tokenizer.save_model(save_path)

['tokenized_data\\vocab.json', 'tokenized_data\\merges.txt']

In [2]:
import tensorflow as tf
import pandas as pd

# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})
# creating the configurations from which the model can be made
config = GPT2Config(
  vocab_size=tokenizer.vocab_size,
  bos_token_id='<s>',
  eos_token_id='</s>'
)
# creating the model
model = TFGPT2LMHeadModel(config)

In [4]:
single_string = ''
for filename in paths:
  with open(filename, "r", encoding='utf-8') as f:
   x = f.read()
  single_string += x + '</s>'
string_tokenized = tokenizer.encode(single_string)

In [44]:
#print(string_tokenized)

In [5]:
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
#print(examples)
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])
  #print(labels)
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

KeyboardInterrupt: 

In [46]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [47]:
dataset

<BatchDataset shapes: ((12, 99), (12, 99)), types: (tf.int32, tf.int32)>

In [48]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

In [40]:
num_epoch = 10
history = model.fit(dataset, epochs=num_epoch)

Epoch 1/10
 11/533 [..............................] - ETA: 41:04 - loss: 6.8251 - logits_loss: 6.8251 - logits_accuracy: 0.3362 - past_key_values_1_accuracy: 0.0082 - past_key_values_2_accuracy: 0.0100 - past_key_values_3_accuracy: 0.0076 - past_key_values_4_accuracy: 0.0052 - past_key_values_5_accuracy: 0.0128 - past_key_values_6_accuracy: 0.0090 - past_key_values_7_accuracy: 0.0154 - past_key_values_8_accuracy: 0.0033 - past_key_values_9_accuracy: 0.0038 - past_key_values_10_accuracy: 0.0047 - past_key_values_11_accuracy: 0.0056 - past_key_values_12_accuracy: 0.0055 

d:\programdata\venv\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
text = "trump has"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# getting out output
beam_output = model.generate(
  input_ids,
  max_length = 50,
  num_beams = 5,
  temperature = 0.7,
  no_repeat_ngram_size=2,
  num_return_sequences=5
)

In [ ]:
print(tokenizer.decode(beam_output[0]))